In [1]:
import json
import re
import datetime
import time

import tweepy
from tweepy import OAuthHandler

In [2]:
consumer_key = 'XXX'          
consumer_secret = 'XXX'   
access_token = 'XXX'           
access_token_secret = 'XXX'  
# create OAuthHandler object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# set access token and secret
auth.set_access_token(access_token, access_token_secret)
# create tweepy API object to fetch tweets
api = tweepy.API(auth)

# data source: https://ieee-dataport.org/open-access/coronavirus-covid-19-geo-tagged-tweets-dataset
id_data = json.load(open('april28-may26.json','r'))
id_sentiment_list = id_data['rows']

In [3]:
id_list = []
for item in id_sentiment_list:
    id_list.append(item[0])

print('Len ID List:',len(id_list),id_list[:10])

Len ID List: 42341 [1254995208888094721, 1254995485452050438, 1254995705527177216, 1254995868333289475, 1254996123514789893, 1254996305082036224, 1254996436162277377, 1254996786105790464, 1254997859285508096, 1254998021722390531]


In [4]:
pat1 = re.compile('[^A-Za-z]')
pat_url = re.compile('http[s]?://\S+')

def hydrate_tweets(ids):
    result = []
    fetched_tweets = api.statuses_lookup(ids,tweet_mode='extended')
    for tweet in fetched_tweets:
        try:
            text = tweet.retweeted_status.full_text
            rt = True
        except AttributeError:  # Not a Retweet
            rt = False
            text = tweet.full_text
        
        created_at = tweet.created_at
        timestamp = int(created_at.replace(tzinfo=datetime.timezone.utc).timestamp())
        obj = {
            'text':tweet.full_text,
            'timestamp':timestamp,
            'tweet_id':tweet.id_str,
            'user_id':tweet.user.id_str,
            'user_followers':tweet.user.followers_count,
            'is_rt':rt,
            #longitude,latitude
            'coordinates':tweet.coordinates,
            'user_location':tweet.user.location,
            'favorites':tweet.favorite_count,
            'retweets':tweet.retweet_count,
            'lang':tweet.lang
        }
        result.append(obj)
    
    return result

In [5]:
i = 0
tweets_fetched = 0
len_list = len(id_list)
all_results = []
try:
    ids_done = set(json.load(open('ids_done.json','r',encoding='utf-8'))['done'])
except:
    ids_done = set()
    
try:
    all_results = json.load(open('tweets.json','r',encoding='utf-8'))['tweets']
except:
    all_results = []
    
while i < len(id_list):
    curr_ids = id_list[i:i+100]
    
    first = curr_ids[0]
    while first in ids_done:
        i += 100
        first = id_list[i]
    curr_ids = id_list[i:i+100]
    
    try:
        print('Current starting position:',i)
        print('Fetching new tweets')
        result = hydrate_tweets(curr_ids)
        ids_done |= set(curr_ids)
        i += 100
        all_results.extend(result)
        json.dump({'tweets':all_results},open('tweets.json','w',encoding='utf-8'))
        json.dump({'done':list(ids_done)}, open('ids_done.json','w',encoding='utf-8'))
        tweets_fetched += len(result)
        print('Total tweets fetched:',tweets_fetched)
        print()
        time.sleep(2)
    except tweepy.error.RateLimitError:
        print('RATE LIMIT ERROR')
        print()
        # Wait for a minute to try again
        time.sleep(60)
    

Current starting position: 25200
Fetching new tweets
Total tweets fetched: 94

Current starting position: 25300
Fetching new tweets
Total tweets fetched: 192

Current starting position: 25400
Fetching new tweets
Total tweets fetched: 290

Current starting position: 25500
Fetching new tweets
Total tweets fetched: 389

Current starting position: 25600
Fetching new tweets
Total tweets fetched: 487

Current starting position: 25700
Fetching new tweets
Total tweets fetched: 584

Current starting position: 25800
Fetching new tweets
Total tweets fetched: 679

Current starting position: 25900
Fetching new tweets
Total tweets fetched: 778

Current starting position: 26000
Fetching new tweets
Total tweets fetched: 877

Current starting position: 26100
Fetching new tweets
Total tweets fetched: 975

Current starting position: 26200
Fetching new tweets
Total tweets fetched: 1074

Current starting position: 26300
Fetching new tweets
Total tweets fetched: 1171

Current starting position: 26400
Fetchi

Total tweets fetched: 9989

Current starting position: 35400
Fetching new tweets
Total tweets fetched: 10086

Current starting position: 35500
Fetching new tweets
Total tweets fetched: 10182

Current starting position: 35600
Fetching new tweets
Total tweets fetched: 10280

Current starting position: 35700
Fetching new tweets
Total tweets fetched: 10377

Current starting position: 35800
Fetching new tweets
Total tweets fetched: 10476

Current starting position: 35900
Fetching new tweets
Total tweets fetched: 10572

Current starting position: 36000
Fetching new tweets
Total tweets fetched: 10667

Current starting position: 36100
Fetching new tweets
Total tweets fetched: 10763

Current starting position: 36200
Fetching new tweets
Total tweets fetched: 10863

Current starting position: 36300
Fetching new tweets
Total tweets fetched: 10960

Current starting position: 36400
Fetching new tweets
Total tweets fetched: 11060

Current starting position: 36500
Fetching new tweets
Total tweets fetc